In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re


In [6]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device in use: {device}')

Device in use: cpu


In [9]:
import re
from collections import Counter

class SimpleTokenizer:
    def __init__(self):
        self.vocab = {}
        self.rev_vocab = {}
        self.token_count = 0

    def bulid_vocab(self, texxts,min_freq=2):
        all_chars=[]
        for text in texts:
            cleaned_text =re.sub(r'[^a-zA-Z0-9\s]', '', text)     
            all_chars.extend(list(cleaned_text))

        char_counts = Counter(all_chars)

        # Add sppecial tokens
        self.vocab= {
            '<pad>': 0,
            '<unk>': 1,
            '<sos>': 2,
            '<eos>': 3
        }
        self.inx_to_token = {0: '<pad>', 1: '<unk>', 2: '<sos>', 3: '<eos>'}

        indx=4
        for char,count in char_counts.items():
            if count >= min_freq:
                self.vocab[char] = indx
                self.inx_to_token[indx] = char
                indx += 1    
            
            def encode(self,token):    
                tokens =[self.vocab['<bos>']]
                for char in text:
                    tokens.append(self.vocab.get(char, self.vocab['<unk>']))
                tokens.append(self.vocab['<eos>'])
                return tokens
            
            def decode(self,token_ids):
                text = ""
                for token_id in token_ids:
                    if token_id in [self.vocab['<pad>'],self.vocab['<bps>'],self.vocab['<eos>']]:
                        continue
                    elif token_id == self.vocab['<unk>']:
                        text += '?'
                    else:
                        text += self.inx_to_token(token_id, '')
                return text


